# NLP wiht LSTM TensorFlow Youtube Tutorial 

Video: https://www.youtube.com/watch?v=ZMudJXhsUpY&list=PLQY2H8rRoyvzDbLUZkbudP-MFQZwNmU4S&index=6

Code: https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb#scrollTo=w9vH8Y59ajYL

## Setup 

In [1]:
import tensorflow as tf
import keras 

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import json

import numpy as np 
import os
import random
import re

#os.environ["TF_USE_LEGACY_KERAS"] = "1" # https://github.com/tensorflow/tfjs/issues/8328#issuecomment-2212490809

! echo $TF_USE_LEGACY_KERAS
! python3 --version
print(f"Tensorflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")


2024-11-09 19:53:41.089167: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-09 19:53:41.626554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-09 19:53:41.628051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-09 19:53:41.711869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 19:53:41.865679: I tensorflow/core/platform/cpu_feature_guar


Python 3.10.15
Tensorflow Version: 2.15.1
Keras Version: 2.15.0


##  Einstellungen 

In [2]:
traning_new_model = True                # Soll ein neues Modell traniert werden ?
copie_js_model_to_react = True          # Soll das js Modell direkt rüber kopiert werden ?

example_text = "Hallo ich bin"
#max_sequence_len=10
text_file = "10k_sentences.txt"
traning_sentences = 500
model_name = "lstm" 

text_file = os.path.join(os.path.abspath("/workspace/dataset"), text_file)
react_model_dir = os.path.abspath("/workspace/webpage/silvan-webpage/public/models")
model_file = os.path.join(os.path.abspath("/workspace/models"), model_name)
js_model_dir = os.path.join(os.path.abspath("/workspace/models"), model_name + "_js")
react_model_dir = os.path.abspath("/workspace/webpage/silvan-webpage/public/models")
word_index_file = os.path.join(model_file, 'tokenizer_word_index.json')

## Data

In [ ]:
def lade_zufaellige_zeilen(dateipfad, traning_sentences):

    # Zeilen zählne
    with open(dateipfad, 'r') as datei:
        anzahl_zeilen = sum(1 for zeile in datei)
    print(f"Text beinhaltet {anzahl_zeilen} Zeile \n")
    
    # Zufällige Zeilennummern generieren
    zufaellige_zeilennummern = set(random.sample(range(anzahl_zeilen), traning_sentences))
    print(f"{anzahl_zeilen} zufällige Zahlen zwischen 1 und {anzahl_zeilen}: {zufaellige_zeilennummern} \n")
    
    # Ausgewählten Zeilen laden
    ausgewaehlte_zeilen = ""
    with open(dateipfad, 'r') as datei:
        for nummer, zeile in enumerate(datei):
            if nummer in zufaellige_zeilennummern:
                zeile = re.sub(r'[^a-zA-Zäöüß.,!? \n]', '', zeile)
                ausgewaehlte_zeilen += zeile.strip().lower() + "\n"
    print(f"Trainingstext: \n{ausgewaehlte_zeilen[:100]}")
    
    return ausgewaehlte_zeilen

# Beispiel-Aufruf der Funktion
data = lade_zufaellige_zeilen(text_file, traning_sentences)


Text beinhaltet 9902 Zeile 

9902 zufällige Zahlen zwischen 1 und 9902: {6145, 2049, 6147, 4, 6, 9, 11, 6158, 6159, 2072, 33, 2083, 39, 4138, 6193, 6200, 8252, 63, 6210, 6214, 8265, 6218, 6231, 2139, 6240, 2149, 101, 103, 8297, 107, 6254, 4206, 2158, 116, 4216, 4219, 4221, 2187, 6285, 148, 156, 165, 170, 177, 4284, 6337, 4290, 4297, 4308, 6356, 6357, 216, 8184, 8412, 4319, 6370, 2275, 6379, 4332, 6385, 4340, 2305, 4359, 8459, 6415, 281, 2329, 2334, 2341, 300, 4399, 8497, 6450, 308, 317, 2366, 8512, 2369, 8515, 8520, 2390, 8534, 346, 8542, 4446, 363, 4461, 8558, 6526, 384, 6534, 6536, 2445, 6544, 2452, 8602, 4507, 6554, 413, 4512, 419, 6564, 6569, 2474, 2477, 6579, 8628, 4536, 2490, 8634, 2494, 2499, 458, 2507, 2514, 8667, 476, 2545, 503, 509, 6656, 8704, 6658, 6664, 8714, 8716, 6669, 4622, 8721, 541, 2598, 552, 8748, 561, 6706, 6708, 6716, 8764, 2620, 6726, 8774, 6734, 8791, 6744, 8794, 603, 2651, 8798, 4703, 6761, 619, 2667, 8813, 623, 8821, 4726, 633, 8825, 4733, 6786, 2694, 647, 884

In [4]:
def erstelle_tokenizer(data, word_index_file):

    # Korpus erstellen und Tokenizer initialisieren
    tokenizer = Tokenizer()
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    print(f"Corpus mit der länge {len(corpus)}: \n{corpus} \n")
    print(f"Der Tokenizer kennt {total_words} Wörter \n")
    print(f"Die Wörter werden folgenden Indexen zugeordnet: \n{tokenizer.word_index}")
   
    
    # Wort-Index-Mapping in JSON-Datei speichern
    os.makedirs(os.path.dirname(word_index_file), exist_ok=True)
    word_index = tokenizer.word_index
    with open(word_index_file, 'w') as f:
        json.dump(word_index, f)

    return corpus, tokenizer, total_words

corpus, tokenizer, total_words = erstelle_tokenizer(data, word_index_file)


Corpus mit der länge 501: 
['aber was ist eigentlich der richtige durstlöscher?', 'aber wer erledigt den schmutzigen job?', 'aber wir wollen ja seriös bleiben.', 'aber wollten sie das?', 'adlershorst ist eine genossenschaft geblieben.', 'hnlich äußerte sich auch der hessische grünenchef tarek alwazir.', 'alle angaben beruhen auf den auskünften des verkäufers.', 'allerdings mit jeweils individuellem bewußtsein.', 'allerdings mit wechselndem erfolg.', 'allerdings nur mit mäßigem erfolg.', 'allerdings weist der bitkom nichtsdestotrotz auf schwächen hin.', 'alquist er soll warten.', 'als grundlage dienten den programmierern deutsche telefonbücher.', 'also freundete ich mich mit einer schwarzen k an.', 'also in der ddr.', 'also wird es eingeschränkt.', 'am freitag wird weiterverhandelt.', 'anders entwickelt sich die situation in den neuen ländern.', 'an fahrzeugen besteht ein absoluter nachholbedarf.', 'angeblich wurden sie massiv bedroht.', 'angst habe nur seine lebensgefährtin.', 'anschli

In [5]:
def generier_daten(corpus, tokenizer, total_words):

    input_sequences = []
    is_first_line = True

    # Wandelt jede Zeile in eine Liste von Wortindizes und generiert n-Gramm-Sequenzen
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        print(f"Satz: {line} tokenisiert zu: {token_list} \n") if is_first_line == True else None

        # n-Gramm-Sequenz generieren
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
            print(f"Generierte n-Gramm-Sequenz: {n_gram_sequence}") if is_first_line == True else None
        
        is_first_line = False

    # Alle Sequences auf gleiche Länge bringen
    max_sequence_len = max([len(seq) for seq in input_sequences])
    print(f"\nLängster Satz: {max_sequence_len}")
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    print(f"Padding mit der Shape {input_sequences.shape} : {input_sequences[0]}")


    # Aufteilen in Prädiktoren (xs) und Labels (ys)
    xs, labels = input_sequences[:,:-1], input_sequences[:,-1]
    ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

    return xs, ys, max_sequence_len

xs, ys, max_sequence_len = generier_daten(corpus, tokenizer, total_words)


Satz: aber was ist eigentlich der richtige durstlöscher? tokenisiert zu: [25, 48, 3, 298, 2, 143, 299] 

Generierte n-Gramm-Sequenz: [25, 48]
Generierte n-Gramm-Sequenz: [25, 48, 3]
Generierte n-Gramm-Sequenz: [25, 48, 3, 298]
Generierte n-Gramm-Sequenz: [25, 48, 3, 298, 2]
Generierte n-Gramm-Sequenz: [25, 48, 3, 298, 2, 143]
Generierte n-Gramm-Sequenz: [25, 48, 3, 298, 2, 143, 299]

Längster Satz: 11
Padding mit der Shape (2763, 11) : [ 0  0  0  0  0  0  0  0  0 25 48]


## Modell

In [6]:
# Modell Struktur 

from tensorflow.keras.models import load_model

def create_model():
    model = Sequential()
    model.add(Embedding(input_dim=total_words,                  # Anzahl aller Wörter
                        output_dim=100,                         # Umwandlung des Wortes (des Integers) in den einen n-dimensionanlen Vektor 
                        input_length=(max_sequence_len-1)))     # Länge der Eingaben (Anzahl Wörter)
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation='softmax'))
    
    # Optimizer und Modell-Compile
    adam = Adam(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Prüfen, ob das Modell bereits existiert
if os.path.exists(model_file) and traning_new_model == False:
    print("Modell gefunden. Lade bestehendes Modell...")
    model = load_model(model_file)  
    
else:
    if traning_new_model ==True:
        print("Neuse Model soll traniert werden")
    else: 
        print("Altes Modell nicht gefunden")
    
    print("Startet Training \n\n")
    model = create_model()  # Neues Modell erstellen
    
    # Modell trainieren
    history = model.fit(xs, ys, epochs=100, verbose=1)
    
    # Modell speichern
    model.save(model_file)
    print(f"Modell gespeichert unter: {model_file}")


Neuse Model soll traniert werden
Startet Training 


Epoch 1/100
87/87 [==============================] - 8s 34ms/step - loss: 7.1851 - accuracy: 0.0297
Epoch 2/100
87/87 [==============================] - 3s 31ms/step - loss: 6.4214 - accuracy: 0.0442
Epoch 3/100
87/87 [==============================] - 3s 30ms/step - loss: 5.7505 - accuracy: 0.0586
Epoch 4/100
87/87 [==============================] - 3s 29ms/step - loss: 4.7143 - accuracy: 0.1252
Epoch 5/100
87/87 [==============================] - 3s 34ms/step - loss: 3.3539 - accuracy: 0.2859
Epoch 6/100
87/87 [==============================] - 3s 29ms/step - loss: 2.0638 - accuracy: 0.5346
Epoch 7/100
87/87 [==============================] - 2s 27ms/step - loss: 1.2482 - accuracy: 0.7315
Epoch 8/100
87/87 [==============================] - 2s 28ms/step - loss: 0.8059 - accuracy: 0.8248
Epoch 9/100
87/87 [==============================] - 3s 30ms/step - loss: 0.6089 - accuracy: 0.8636
Epoch 10/100
87/87 [===========================

INFO:tensorflow:Assets written to: /workspace/models/lstm/assets


Modell gespeichert unter: /workspace/models/lstm


In [7]:
# Zugriff auf Embedding-Schicht
embedding_layer = model.layers[0]  # Die erste Schicht ist die Embedding-Schicht

# Eingabedimensionen (Vokabulargröße) und Ausgabedimensionen (Embedding-Dimension)
embedding_input_dim = embedding_layer.input_dim                 # Anzahl der Wörter im Vokabular
embedding_input_shape = embedding_layer.input_shape             # Form der Eingabedaten (z.B. Batchsize, Satzlänge)
embedding_output_shape = embedding_layer.output_shape           # Form der Ausgabedaten (z.B. Batchsize, Satzlänge, Embedding-Dimension)

embedding_weights = embedding_layer.get_weights()               # Zugriff auf die Gewichte der Embedding-Schicht
embedding_output_dim = embedding_layer.output_dim               # Anzahl der Dimensionen für jedes Wort (z.B. 100 für 100-Dimensionales Embedding)


# Ausgabe der Informationen
print(f"Embedding Eingabedimensionen: {embedding_input_dim}")
print(f"Form der Eingabedaten: {embedding_input_shape}")  # Form der Eingabedaten
print(f"Form der Ausgabedaten: {embedding_output_shape}")  # Form der Ausgabedaten

print(f"Embedding Gewichtedimensionen: {embedding_weights[0].shape}")  # Gewichtedimensionen der Embedding-Matrix
print(f"Embedding Ausgabedimensionen: {embedding_output_dim}")


Embedding Eingabedimensionen: 1697
Form der Eingabedaten: (None, 10)
Form der Ausgabedaten: (None, 10, 100)
Embedding Gewichtedimensionen: (1697, 100)
Embedding Ausgabedimensionen: 100


## Verwendung 

### Text generieren

In [8]:
next_words = 10
seed_text = example_text

for _ in range(next_words):
    # Tokenisiere den aktuellen Seed-Text und führe Padding durch
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    
    # Mache eine Vorhersage basierend auf dem aktuellen Seed-Text
    predicted_probs = model.predict(token_list, verbose=0)[0]
    predicted = np.argmax(predicted_probs)  # Bestimme das Wort mit der höchsten Wahrscheinlichkeit
    
    # Finde das vorhergesagte Wort im Tokenizer-Wörterbuch
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    
    # Wenn das Wort gefunden wurde, erweitere den Seed-Text
    if output_word:
        seed_text += " " + output_word
    else:
        print("Kein Wort gefunden für den vorhergesagten Index.")
        break

# Ausgabe des vollständigen Textes
print("Erweiterter Text:", seed_text)


Erweiterter Text: Hallo ich bin der größte softie an die dm weiter zu statt deformation


### Nextes Word vorhesagen  

In [9]:
import numpy as np

top_k = 10  # Anzahl der Top-Wörter
seed_text = "Ich"

# Tokenisierung und Padding des Eingabetexts
token_list = tokenizer.texts_to_sequences([seed_text])[0]
print(f"Eingabe: '{seed_text}' tokeiziert zu: {token_list}")
token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
print(f"Mit Paddig: {token_list}")

# Vorhersagen berechnen
predictions = model.predict(token_list, verbose=0)[0]  # Softmax-Ausgabe

# Top-10 wahrscheinlichste Wörter finden
top_indices = np.argsort(predictions)[-top_k:][::-1]  #Sortiere absteigend

# Wörter und Wahrscheinlichkeiten sammeln
top_words = [(word, predictions[index]) for word, index in tokenizer.word_index.items() if index in top_indices]

# Sortiere die Wörter nach Wahrscheinlichkeit in absteigender Reihenfolge
top_words = sorted(top_words, key=lambda x: x[1], reverse=True)

# Ausgabe der Top-10 Wörter und Wahrscheinlichkeiten
print(f"\nTop {top_k} nächste Wörter für '{seed_text}':")
for word, prob in top_words:
    print(f"{word}: {(prob * 100):.4f} %")


Eingabe: 'Ich' tokeiziert zu: [26]
Mit Paddig: [[ 0  0  0  0  0  0  0  0  0 26]]

Top 10 nächste Wörter für 'Ich':
plädiere: 15.8889 %
kenne: 15.1063 %
werde: 15.0562 %
wähle: 14.7281 %
bin: 13.7550 %
tippe: 10.8042 %
nenne: 10.6282 %
wird: 0.2152 %
den: 0.1665 %
einen: 0.1265 %


## Konvertierung 

In [10]:
if traning_new_model:
    print("Konveriert neues Modell \n\n")
    #! tensorflowjs_converter --input_format=keras {model_file} {js_model_dir}
    ! tensorflowjs_converter --input_format=tf_saved_model {model_file} {js_model_dir}
else:
    print("Konvertiertes Model bereits vorhanden")

print("Kopiert Tokenizer")
! cp -r {word_index_file} {js_model_dir}


Konveriert neues Modell 


2024-11-09 19:58:28.484648: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-09 19:58:28.484734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-09 19:58:28.485841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-09 19:58:29.710243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Kopiert Tokenizer


In [11]:
if copie_js_model_to_react:
    print("Kopiere JS-Model in React-Model")
    ! cp -r {js_model_dir} {react_model_dir}
else:
    print("JS-Model bereits im React-Model vorhanden")

Kopiere JS-Model in React-Model
